In [1]:
#The goal is to Start a sushi-restaurant in the city of Brussels
#First we'll look for all current sushi-restaurants and decide on possible locations where there are not many sushi restaurants around
    #For this step we'll use foursqaure to search for specific venue's
    #The information will be processed into a dataframe and plotted onto a map

#We'll then check for the rating of the restaurants near optimal locations
    #For this step we'll use foursquare again to list the ratings of specific restaurant
    #The restaurants will be grouped into clusters based on their rating

#Finally a location can be chosen based on the final map

In [2]:
#Importing and installing necessary libraries

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [3]:
#Adding Foursquare information to variables:
CLIENT_ID = 'IL2QNRRCV02PKZRV1WPPAEXVML4XABBERFDZOTCMY4FWRWAY' # your Foursquare ID
CLIENT_SECRET = 'REMOVED' # your Foursquare Secret
VERSION = '20190925'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: IL2QNRRCV02PKZRV1WPPAEXVML4XABBERFDZOTCMY4FWRWAY


In [4]:
#Getting latitude and longtitude of Brussels
address = 'City of Brussels, Belgium'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

50.81977585 4.39879936804723


In [6]:
search_query = 'sushi'
radius = 5700
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
Brussel_data = json_normalize(venues)
Brussel_data.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4fbf8ea6e4b0096e866e4cbb,Rue de Tervaetestraat 12,BE,Brussel,België,NaN,1337,"[Rue de Tervaetestraat 12, 1040 Brussel, België]","[{'label': 'display', 'lat': 50.83177323376235...",50.831773,4.399853,NaN,1040,Brussels Hoofdstedelijk Gewest,Oma Sushi Foods,v-1569418556
1,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,50e87fbee4b0aa6efde89da4,Kroonlaan 431/2 Avenue de la Couronne,BE,Elsene,België,NaN,595,"[Kroonlaan 431/2 Avenue de la Couronne, 1050 E...","[{'label': 'display', 'lat': 50.81661993550632...",50.816620,4.391963,NaN,1050,Brussels Hoofdstedelijk Gewest,Sushi House,v-1569418556
2,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,5c1a4a789cadd9002b394876,Rue Defacqz 85,BE,Brussel,België,NaN,3027,"[Rue Defacqz 85, 1060 Brussel, België]","[{'label': 'display', 'lat': 50.826938, 'lng':...",50.826938,4.357260,Chatelain,1060,Brussels Hoofdstedelijk Gewest,Sushi Ruko,v-1569418556
3,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,5592913c498efb6e6f278f62,Avenue Orban 244,BE,",Woluwé-Saint-Pierre",België,NaN,5109,"[Avenue Orban 244, 1150 ,Woluwé-Saint-Pierre, ...","[{'label': 'display', 'lat': 50.83937975776315...",50.839380,4.464500,NaN,1150,NaN,Sushi Shop Stockel,v-1569418556
4,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,579f9897cd102f2ed97ee6c5,Avenue Orban 235,BE,Sint-Pieters-Woluwe,België,NaN,5112,"[Avenue Orban 235, 1150 Sint-Pieters-Woluwe, B...","[{'label': 'display', 'lat': 50.839304516978, ...",50.839305,4.464604,NaN,1150,Brussels Hoofdstedelijk Gewest,Tenshi Sushi,v-1569418556


In [8]:
Brussel_data.shape

(30, 18)

In [11]:
#removing possible NaN entries
Brussel_data.dropna(subset = ['location.city'], inplace = True)

In [12]:
#Getting only sushi restaurants located in Brussel by removing outsiders
    #Final dataset before visualisation
Brussel_data = Brussel_data[Brussel_data['location.city'].str.contains('Brussel')].reset_index(drop=True)
Brussel_data.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4fbf8ea6e4b0096e866e4cbb,Rue de Tervaetestraat 12,BE,Brussel,België,NaN,1337,"[Rue de Tervaetestraat 12, 1040 Brussel, België]","[{'label': 'display', 'lat': 50.83177323376235...",50.831773,4.399853,NaN,1040,Brussels Hoofdstedelijk Gewest,Oma Sushi Foods,v-1569418556
1,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,5c1a4a789cadd9002b394876,Rue Defacqz 85,BE,Brussel,België,NaN,3027,"[Rue Defacqz 85, 1060 Brussel, België]","[{'label': 'display', 'lat': 50.826938, 'lng':...",50.826938,4.357260,Chatelain,1060,Brussels Hoofdstedelijk Gewest,Sushi Ruko,v-1569418556
2,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,59ae7adf86bc49021b92145e,Avenue Fonsny 47,BE,Brussel,België,NaN,4645,"[Avenue Fonsny 47, 1060 Brussel, België]","[{'label': 'display', 'lat': 50.836434, 'lng':...",50.836434,4.338221,Hallepoort,1060,Brussels Hoofdstedelijk Gewest,Sushi Gourmet,v-1569418556
3,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,4b3bd484f964a5200d7c25e3,Louizalaan 144 Avenue Louise,BE,Brussel,België,NaN,2808,"[Louizalaan 144 Avenue Louise, 1050 Brussel, B...","[{'label': 'display', 'lat': 50.82929312624903...",50.829293,4.361817,NaN,1050,Brussels Hoofdstedelijk Gewest,Sushi Shop,v-1569418556
4,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,525542db11d2d0218e33aaa0,Galerie Ravenstein 29,BE,Brussel,België,NaN,3880,"[Galerie Ravenstein 29, 1000 Brussel, België]","[{'label': 'display', 'lat': 50.84324570388764...",50.843246,4.357986,NaN,1000,Brussels Hoofdstedelijk Gewest,Kaiten Sushi Bar,v-1569418556


In [14]:
#Final dataset dimensions
Brussel_data.shape

(16, 18)